In [58]:
import pandas as pd
import datetime as dt
import numpy as np
import win32com.client
import warnings

warnings.filterwarnings("ignore")

# Начато 21.01.2022 в 19:17
ROW_DATA_PATH = "RawData/"
ROW_FILENAME = "2022-01-31 Январь 2022.xlsx"
# ROW_FILENAME = "2022-01-25 Январь-Декабрь.xlsx"
ROW_DATA_COLUMNS = {
                    "Дата" : "FDate",
                    "Функциональное направление" : "FNRow",
                    "МВЗ" : "MVZ",
                    "Направление" : "DivisionRow",
                    "Подразделение" : "SubDivision",
                    "Пользователь" : "User",
                    "Северный работник" : "Northern",
                    "Проект" : "Project",
                    "Статус проекта" : "ProjectState",
                    "Менеджер проекта" : "ProjectManager",
                    "Вид проекта" : "ProjectType",
                    "Кол-во штатных единиц" : "KoBo",
                    "План, FTE" : "PlanFTE",
                    "Договор" : "Contract",
                    "Фактические трудозатраты (час.) (Сумма)" : "FactHour",
                    "Unnamed: 15" : "Unnamed15",
                    "Unnamed: 16" : "Unnamed16"
                    }

RESULT_DATA_COLUMNS = [
                        "FDate",
                        "Month",
                        "FN",
                        "Division",
                        "User",
                        "Project",
                        "ProjectType",
                        "ProjectSubType",
                        "ProjectManager", 
                        "PlanFTE",
                        "FactFTE",
                        "Pdr_Proj_User_Month",
                        "Pdr_User_Proj_Month",
                        "Pdr_User_Month",
                        "Pdr_Proj_Month",
                        "ProjMang_Proj_Month",
                        "ProjMang_Proj_User_Month"
                      ]
ROW_DATA_DROP_COLUMNS = ["MVZ", "KoBo", "Contract", "Unnamed15", "Unnamed16"]
DONT_REPLACE_ENTER = ["Month"]

BOOLEAN_VALUES_SUBST = {"ЛОЖЬ": 0, "ИСТИНА": 1}

PARAMETERS_SECTION_NAME = "Parameters"
ROW_DATA_SECTION_NAME = "RawDataPath"
ROUND_FTE_SECTION_NAME = "RoundFTE"

MONTH_WORKING_HOURS_TABLE = "WHours.xlsx"
DIVISIONS_NAMES_TABLE = "ShortDivisionNames.xlsx"
FNS_NAMES_TABLE = "ShortFNNames.xlsx"
PROJECTS_SUB_TYPES_TABLE = "ProjectsSubTypes.xlsx"

ROUND_FTE_VALUE = 0
FACT_IS_PLAN_MARKER = "(факт=плану)"
OTHER_PROJECT_SUB_TYPE = "_Прочее"

p_delete_vacation = True

In [59]:
def get_parameter_value(paramname):
    # Читаем настройки
    settings_df = pd.read_excel("Settings.xlsx", engine='openpyxl')
    settings_df.dropna(how='all', inplace=True)
    ret_value = settings_df[settings_df["ParameterName"]==paramname]["ParameterValue"].to_list()[0]
    if type(ret_value) == str:
        ret_value = ret_value.replace("\\", "/")
        if ret_value[-1] == "/":
            ret_value = ret_value[:-1:]
    
    return (ret_value)

In [60]:
import os
os.path.join(os.path.join(os.getcwd(), "c:\\tmp"), "Settings.xlsx")

'c:\\tmp\\Settings.xlsx'

In [61]:
def load_row_data():
    # Загружаем сырые данные
    df = pd.read_excel(get_parameter_value(ROW_DATA_SECTION_NAME) + "/" + ROW_FILENAME, engine='openpyxl')
    df.dropna(how='all', inplace=True)
    df.rename(columns=ROW_DATA_COLUMNS, inplace=True)
    exist_drop_columns_list = list(set(ROW_DATA_DROP_COLUMNS) & set(df.dtypes.keys()))
    df.drop(columns=exist_drop_columns_list, inplace=True)

    return (df)

In [62]:
def load_parameter_table(tablename):
    # Загружаем соответствующу таблицу с параметрами
    parameter_df = pd.read_excel(get_parameter_value(PARAMETERS_SECTION_NAME) + "/" + tablename, engine='openpyxl')
    parameter_df.dropna(how='all', inplace=True)
    
    return(parameter_df)

In [63]:
def udata_2_date(data):
    if (data != data):
        ret_date = data
    elif type(data) == str:
        ret_date = dt.datetime.strptime("01."+data, '%d.%m.%Y')
    elif type(data) == float:
        ret_date = dt.datetime.strptime("01."+str(data), '%d.%m.%Y')
    else:
        ret_date = data
    
    return(ret_date)

In [64]:
def calc_fact_fte(FactHour, Northern, CHour, NHour, Project, PlanFTE):
    if Project.find(FACT_IS_PLAN_MARKER) >= 0:
        fact_fte = PlanFTE
    else:
        month_hours = NHour if Northern else CHour
        fact_fte = round(FactHour / month_hours, ROUND_FTE_VALUE)
    return(fact_fte)

In [65]:
def add_combine_columns(df):
# "Month",
# "FN",
# "Division",
# "User",
# "Project",
# "ProjectType",
# "ProjectSubType",
# "PlanFTE",
# "FactFTE"
# 
# Подразделение + Проект + ФИО + Месяц
# Подразделение + ФИО + Проект + Месяц
# Подразделение + ФИО + Месяц
# Подразделение + Проект + Месяц
# ПМ + Проект + Месяц
# ПМ + Проект + ФИО + Месяц
    df["ShortProject"] = df["Project"].str[:7]
    df["Pdr_Proj_User_Month"] = df["Division"] +"#"+ df["ShortProject"] +"#"+ df["User"] +"#"+ df["Month"]
    df["Pdr_User_Proj_Month"] = df["Division"] +"#"+ df["User"] +"#"+ df["ShortProject"] +"#"+ df["Month"]
    df["Pdr_User_Month"] = df["Division"] +"#"+ df["User"] +"#"+ df["Month"]
    df["Pdr_Proj_Month"] = df["Division"] +"#"+ df["ShortProject"] +"#"+ df["Month"]
    df["ProjMang_Proj_Month"] = df["ProjectManager"] +"#"+ df["ShortProject"] +"#"+ df["Month"]
    df["ProjMang_Proj_User_Month"] = df["ProjectManager"] +"#"+ df["ShortProject"] +"#"+ df["User"] +"#"+ df["Month"]
    
    

In [66]:
def prepare_data(data_df):
    for column_name in set(data_df.dtypes.keys()) - set(DONT_REPLACE_ENTER):
        if data_df.dtypes[column_name] == type(str):
            data_df[column_name] = data_df[column_name].str.replace("\n", "")
    
    data_df["FDate"] = data_df["FDate"].apply(lambda param: udata_2_date(param))
    data_df['Northern'].replace(BOOLEAN_VALUES_SUBST, inplace=True)
    data_df = data_df.merge(month_hours_df, left_on="FDate", right_on="FirstDate", how="inner")
    
    data_df["FactFTE"] = \
        data_df[["FactHour", "Northern", "CHour", "NHour", "Project", "PlanFTE"]].apply( \
            lambda param: calc_fact_fte(*param), axis=1
        )

    data_df = data_df.merge(divisions_names_df, left_on="DivisionRow", right_on="FullDivisionName", how="left")
    data_df["Division"] = data_df[["ShortDivisionName", "DivisionRow"]].apply(lambda param: param[1] if pd.isna(param[0]) else param[0], axis=1)

    data_df = data_df.merge(fns_names_df, left_on="FNRow", right_on="FullFNName", how="left")
    data_df["FN"] = data_df[["ShortFNName", "FNRow"]].apply(lambda param: param[1] if pd.isna(param[0]) else param[0], axis=1)
    
    data_df["ProjectType"] = \
        data_df[["Project", "ProjectType"]].apply( \
            lambda param: "S" if param[0].find(FACT_IS_PLAN_MARKER)>=0 else param[1], axis=1)

    data_df = data_df.merge(projects_sub_types_df, left_on="Project", right_on="ProjectName", how="left")
    data_df["ProjectSubType"] = \
        data_df[["ProjectType", "ProjectSubTypePart"]].apply( \
            lambda param: param[0]+OTHER_PROJECT_SUB_TYPE if pd.isna(param[1]) else param[1], axis=1 \
                                                        )
    
    if p_delete_vacation:
        data_df["User"] = \
            data_df["User"].apply(lambda param: "вакансия" if param.replace(" ","").lower()[:8]=="вакансия" else param)
        
        data_df = data_df[data_df["User"]!="вакансия"]
   
    add_combine_columns(data_df)
    
    return(data_df[RESULT_DATA_COLUMNS])
    return(data_df)
    


In [67]:
row_df = load_row_data()
month_hours_df = load_parameter_table(MONTH_WORKING_HOURS_TABLE)
divisions_names_df = load_parameter_table(DIVISIONS_NAMES_TABLE)
fns_names_df = load_parameter_table(FNS_NAMES_TABLE)
projects_sub_types_df = load_parameter_table(PROJECTS_SUB_TYPES_TABLE)

ROUND_FTE_VALUE = get_parameter_value(ROUND_FTE_SECTION_NAME)

In [68]:
report_df = prepare_data(row_df.copy())


In [69]:
# report_df["FDate"] = report_df["FDate"].dt.tz_localize(tz='Europe/London')
# report_df["FDate"] = pd.to_datetime(report_df["FDate"])
# report_df["FDate"] = report_df["FDate"].astype(str)
report_df["FDate"] = report_df["FDate"].dt.strftime('%Y_%m')
report_df.head()

,FDate,Month,FN,Division,User,Project,ProjectType,ProjectSubType,ProjectManager,PlanFTE,FactFTE,Pdr_Proj_User_Month,Pdr_User_Proj_Month,Pdr_User_Month,Pdr_Proj_Month,ProjMang_Proj_Month,ProjMang_Proj_User_Month
0,2022_01,янв,HCM,ДИСУ,Фролова Елена Евгеньевна,С0010-БОСС-Кадровик ЯКОБС,С,С_Прочее,Пуромов Евгений Владимирович,0.0,0.078,ДИСУ#С0010-Б#Фролова Елена Евгеньевна#янв,ДИСУ#Фролова Елена Евгеньевна#С0010-Б#янв,ДИСУ#Фролова Елена Евгеньевна#янв,ДИСУ#С0010-Б#янв,Пуромов Евгений Владимирович#С0010-Б#янв,Пуромов Евгений Владимирович#С0010-Б#Фролова Е...
1,2022_01,янв,СКК,НСКК,Климова Наталья Геннадьевна,"С0013-EAE.LIMS в ООО ""ИНТЕСМО""",С,С_Прочее,Колисниченко Елена Анатольевна,0.0,0.109,НСКК#С0013-E#Климова Наталья Геннадьевна#янв,НСКК#Климова Наталья Геннадьевна#С0013-E#янв,НСКК#Климова Наталья Геннадьевна#янв,НСКК#С0013-E#янв,Колисниченко Елена Анатольевна#С0013-E#янв,Колисниченко Елена Анатольевна#С0013-E#Климова...
2,2022_01,янв,ГиД,НПИС,Тягун Анна Анатольевна,Т0014-Информационная система оперативного хран...,Т,Т_Прочее,Заславская Ярослава Владимировна,0.0,0.812,НПИС#Т0014-И#Тягун Анна Анатольевна#янв,НПИС#Тягун Анна Анатольевна#Т0014-И#янв,НПИС#Тягун Анна Анатольевна#янв,НПИС#Т0014-И#янв,Заславская Ярослава Владимировна#Т0014-И#янв,Заславская Ярослава Владимировна#Т0014-И#Тягун...
3,2022_01,янв,ГиД,ДКиПС,Алифанова Мария Алексеевна,С0015-Информационная система оперативного хран...,С,С_Прочее,Заславская Ярослава Владимировна,0.0,1.000,ДКиПС#С0015-И#Алифанова Мария Алексеевна#янв,ДКиПС#Алифанова Мария Алексеевна#С0015-И#янв,ДКиПС#Алифанова Мария Алексеевна#янв,ДКиПС#С0015-И#янв,Заславская Ярослава Владимировна#С0015-И#янв,Заславская Ярослава Владимировна#С0015-И#Алифа...
4,2022_01,янв,ГиД,ДКиПС,Карабань Денис Владимирович,С0015-Информационная система оперативного хран...,С,С_Прочее,Заславская Ярослава Владимировна,0.0,1.000,ДКиПС#С0015-И#Карабань Денис Владимирович#янв,ДКиПС#Карабань Денис Владимирович#С0015-И#янв,ДКиПС#Карабань Денис Владимирович#янв,ДКиПС#С0015-И#янв,Заславская Ярослава Владимировна#С0015-И#янв,Заславская Ярослава Владимировна#С0015-И#Караб...


In [70]:
oExcel = win32com.client.Dispatch("Excel.Application")

In [71]:
wb = oExcel.Workbooks.Open("C:/tmp/test/report2.xlsx")
oExcel.visible = False

#sheet = wb.ActiveSheet


In [72]:
oExcel.visible = True

In [73]:
wb.Sheets["ИсходныеДанные"].Activate()
oSheet = wb.Sheets["ИсходныеДанные"]


In [74]:
wb.Sheets["ИсходныеДанные"].Cells(1,1).Value.replace(" ", "")

'Первыйденьмесяца'

In [75]:
oExcel.WorkBooks.Count

3

In [52]:
columns_list = report_df.columns.tolist()
columns_list

['FDate',
 'Month',
 'FN',
 'Division',
 'User',
 'Project',
 'ProjectType',
 'ProjectSubType',
 'ProjectManager',
 'PlanFTE',
 'FactFTE',
 'Pdr_Proj_User_Month',
 'Pdr_User_Proj_Month',
 'Pdr_User_Month',
 'Pdr_Proj_Month',
 'ProjMang_Proj_Month',
 'ProjMang_Proj_User_Month']

In [56]:
report_df.select_dtypes(include = ['object']).columns.tolist()

['FDate',
 'Month',
 'FN',
 'Division',
 'User',
 'Project',
 'ProjectType',
 'ProjectSubType',
 'ProjectManager',
 'Pdr_Proj_User_Month',
 'Pdr_User_Proj_Month',
 'Pdr_User_Month',
 'Pdr_Proj_Month',
 'ProjMang_Proj_Month',
 'ProjMang_Proj_User_Month']

In [50]:
[type(columns_types_list[col_name]) for col_name in columns_list]

[numpy.dtype,
 numpy.dtype,
 numpy.dtype,
 numpy.dtype,
 numpy.dtype,
 numpy.dtype,
 numpy.dtype,
 numpy.dtype,
 numpy.dtype,
 numpy.dtype,
 numpy.dtype,
 numpy.dtype,
 numpy.dtype,
 numpy.dtype,
 numpy.dtype,
 numpy.dtype,
 numpy.dtype]

In [33]:
report_df.dtypes.keys()

Index(['FDate', 'Month', 'FN', 'Division', 'User', 'Project', 'ProjectType',
       'ProjectSubType', 'ProjectManager', 'PlanFTE', 'FactFTE',
       'Pdr_Proj_User_Month', 'Pdr_User_Proj_Month', 'Pdr_User_Month',
       'Pdr_Proj_Month', 'ProjMang_Proj_Month', 'ProjMang_Proj_User_Month'],
      dtype='object')

In [188]:
%%time
row_counter = 0
first_row_with_del = 0
p_found_first_row = False
last_row_4_test = 10000
range_from_excel = wb.Sheets["Отчет"].Range(wb.Sheets["Отчет"].Cells(1,1), wb.Sheets["Отчет"].Cells(last_row_4_test,1)).Value

# Ищем первый признак 'delete'


for row_counter in range(len(range_from_excel)):
    row_del_flag_value = range_from_excel[row_counter][0]
    if row_del_flag_value == None:
        p_found_first_row = False
        break
    
    row_del_flag_value = row_del_flag_value.replace(" ","")
    if row_del_flag_value == "delete":
        p_found_first_row = True
        break

print(row_counter)
        
if row_del_flag_value:
    first_row_with_del = row_counter + 1
    last_row_with_del = row_counter
    while row_counter < len(range_from_excel):
        row_del_flag_value = range_from_excel[last_row_with_del][0]
        if row_del_flag_value == None or row_del_flag_value.replace(" ","") != "delete":
            break
        last_row_with_del += 1

print(first_row_with_del, last_row_with_del, p_found_first_row)



9
10 1000 True
Wall time: 57.5 ms


In [191]:
wb.Sheets["Отчет"].Range(wb.Sheets["Отчет"].Cells( \
            first_row_with_del, 1), wb.Sheets["Отчет"].Cells(last_row_with_del, 1)).Rows.EntireRow.Delete()

True

In [190]:
wb.Sheets["Отчет"].Rows(first_row_with_del, last_row_with_del).Delete()

com_error: (-2147352567, 'Ошибка.', (0, None, None, None, 0, -2146827284), None)